This notebook downloads ZTF DR's light curves, using the ZTF API. 
For more details about ZTF service, go to their documentation: https://irsa.ipac.caltech.edu/data/ZTF/docs/releases/dr08/ztf_release_notes_dr08.pdf
See section "iii. Querying Lightcurves using the API"

More details can be found here: https://irsa.ipac.caltech.edu/docs/program_interface/ztf_lightcurve_api.html

Important notes:
1) Lightcurves were constructed using exclusively measurements from the calibrated single-exposure PSF-fit- derived catalogs, not the aperture-based photometry catalogs.

2) The photometry catalogs, including lightcurve products, contain measurements of both point-like and extended objects. The PSF-fit photometry measurements are optimal for point-sources only. Concentric aperture photometry measurements are more optimal for extended objects. 

3) When downloading  light curves for a particular object, the ZTF API can return more than one light curve in a given filter. This happens because ZTF treats independently the light curves observed in a particular field, filter, and CCD-quadrant; thus, if a source appears in more than one combination of these three, ZTF will provide more than one light curve. The ZTF quadrants are calibrated independently, thus combining light curves from different fields and CCD-quadrants (but same filter), can produce spurious variability (van Roestel et al. 2021). Therefore, to avoid having multiple light curves for any particular source, we recommend to only use the longest light curve (in terms of its timespan or number of epochs) of each filter of a particular object.

4) If you don't specify the data release you want to use, th API will provide you the latest DR. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from matplotlib.pyplot import cm

In [4]:
dr_path = '../datasets/ZTF_DR_data/' #where to save the DR data in csv format, original data

In [6]:
sources = pd.read_csv('../catalogs/sample') #replace here the name of your catalog. 
#It has to contain the following columns: object name, ra and dec. In this example name = SDSS
sources.head()

,_RAJ2000,_DEJ2000,SDSS,RAJ2000,DEJ2000,z,logMBH,e_logMBH,logLbol,LogREdd
0,32.779693,-6.617226,021107.12-063702.0,32.779693,-6.617226,0.628000,8.414858,0.136733,45.503605,-1.051131
1,10.488327,13.437778,004157.19+132616.0,10.488327,13.437778,0.622000,7.570413,0.191406,44.863545,-0.846747
2,7.779218,-4.411826,003107.01-042442.5,7.779218,-4.411826,0.642369,8.696042,0.131428,45.292815,-1.543106
3,36.188918,-3.919286,022445.34-035509.4,36.188918,-3.919286,0.606000,8.420681,0.050707,45.640753,-0.919807
4,11.435759,-3.619997,004544.58-033711.9,11.435759,-3.619997,0.624537,8.049832,0.142181,45.991958,-0.197752


In [13]:
#function to download an individual light curve. This saves the light curve in a csv file
#Possible values for "band" are "g", "r", and "i"

def get_ZTF_DR_lc(name,ra, dec, band, keep_all_columns = False):
    
    if os.path.exists(dr_path+name+'_DR_'+band+'band.csv'):#this check whether the light curve is already available in your local folder
        df = pd.read_csv(dr_path+name+'_DR_'+band+'band.csv')
    else:
        cmd ='wget -O '+dr_path+name+'_DR_'+band+'band.csv "/nph_light_curves?POS=CIRCLE '+str(ra)+' '+str(dec)+' 0.000277778&BANDNAME='+band+'&FORMAT=csv"'
        os.system(cmd)
        df = pd.read_csv(dr_path+name+'_DR_'+band+'band.csv')
            
    if keep_all_columns: print("DR lc for object %s in band %s saved" % (name,band))
    
    else:
        
        #here we select the most important columns.     
        df=df[['oid','mjd','mag','magerr','catflags','filtercode','ra','dec']]

        df.to_csv(dr_path+name+'_DR_'+band+'band.csv', index=False)
        print("DR lc for object %s in band %s saved" % (name,band))

    return(df)

In [10]:
for i in range(len(sources.RAJ2000.values)):#use the proper names for the name, RA, and dec columns from your catalog
    try:
        lcg= get_ZTF_DR_lc(sources.SDSS.values[i],sources.RAJ2000.values[i], sources.DEJ2000.values[i], 'g')
    except:
        print("unsuccessful processing of %s in g band" % (sources.SDSS.values[i])) #the light curve is probably not available in the ZTF DRs
    try:
        lcr= get_ZTF_DR_lc(sources.SDSS.values[i],sources.RAJ2000.values[i], sources.DEJ2000.values[i], 'r')
    except:
        print("unsuccessful processing of %s in r band" % (sources.SDSS.values[i])) #the light curve is probably not available in the ZTF DRs
    try:
        lci= get_ZTF_DR_lc(sources.SDSS.values[i],sources.RAJ2000.values[i], sources.DEJ2000.values[i], 'i')
    except:
        print("unsuccessful processing of %s in i band" % (sources.SDSS.values[i])) #the light curve is probably not available in the ZTF DRs
        
    

DR lc for object 021107.12-063702.0 in band g saved
DR lc for object 021107.12-063702.0 in band r saved
DR lc for object 021107.12-063702.0 in band i saved
DR lc for object 004157.19+132616.0 in band g saved
DR lc for object 004157.19+132616.0 in band r saved
DR lc for object 004157.19+132616.0 in band i saved
DR lc for object 003107.01-042442.5 in band g saved
DR lc for object 003107.01-042442.5 in band r saved
DR lc for object 003107.01-042442.5 in band i saved
DR lc for object 022445.34-035509.4 in band g saved
DR lc for object 022445.34-035509.4 in band r saved
DR lc for object 022445.34-035509.4 in band i saved
DR lc for object 004544.58-033711.9 in band g saved
DR lc for object 004544.58-033711.9 in band r saved
DR lc for object 004544.58-033711.9 in band i saved
DR lc for object 005653.05+010614.4 in band g saved
DR lc for object 005653.05+010614.4 in band r saved
DR lc for object 005653.05+010614.4 in band i saved
DR lc for object 012836.38-024756.2 in band g saved
DR lc for ob

In [ ]:
cmd ='wget -O '+dr_path+name+'_DR_'+band+'band.csv "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE '+str(ra)+' '+str(dec)+' 0.000277778&BANDNAME='+band+'&FORMAT=csv"'

## Download ZTF light curves for RM Objects

In [4]:
objs = pd.read_csv('../datasets/catalogs/rm_objs_catalog').to_numpy()
print(objs)

[['Mrk 876' 824115200000626 243.4882407 65.719442]
 ['Mrk 817' 822203400011253 219.0920037 58.7942865]
 ['3C 273' 473301200007195 187.277924 2.052417]
 ['3C 120' 1503204100005485 68.2962272 5.3543394]
 ['NGC 5548' 1625214300023755 214.4981005 25.1368496]
 ['Mrk 142' 1789210300014255 156.3803313 51.6763677]
 ['NGC 2617' 414306300006175 128.9116496 -4.0882805]
 ['MCG+08-11-011' 1744115300009767 88.7233757 46.4393591]
 ['H 2106-099' 1435104300005070 317.2915353 -9.6707429]]


In [18]:
dr_path = '../datasets/ZTF_rm/' 

for obj in objs:
    ra = obj[2]
    dec = obj[3]
    name = str(obj[0].replace(' ',''))
    get_ZTF_DR_lc(name,ra,dec,'i', keep_all_columns = False)
    get_ZTF_DR_lc(name,ra,dec, 'g', keep_all_columns = False)
    get_ZTF_DR_lc(name,ra,dec,'r', keep_all_columns = False)
    

--2023-02-16 21:50:12--  https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE%20243.4882407%2065.719442%200.000277778&BANDNAME=i&FORMAT=csv
Resolving irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)... 134.4.54.110
Connecting to irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)|134.4.54.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./ZTF_special/Mrk876_DR_iband.csv’

     0K .......... .......... .......... .......... ..........  319K
    50K .......... ..........                                  45.3M=0.2s

2023-02-16 21:50:15 (447 KB/s) - ‘./ZTF_special/Mrk876_DR_iband.csv’ saved [72006]

--2023-02-16 21:50:15--  https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE%20243.4882407%2065.719442%200.000277778&BANDNAME=g&FORMAT=csv


DR lc for object Mrk876 in band i saved


Resolving irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)... 134.4.54.110
Connecting to irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)|134.4.54.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./ZTF_special/Mrk876_DR_gband.csv’

     0K .......... .......... .......... .......... ..........  313K
    50K .......... .......... .......... .......... ..........  668K
   100K .......... .......... .......... .......... .......... 11.1M
   150K .......... .......... .......... .......... ..........  155M
   200K .......... .......... .......... .......... ..........  629K
   250K .......... .......... .......... .......... .......... 44.5M
   300K .......... .......... .......... .......... .......... 89.9M
   350K .......... .......... .......... .......... .......... 65.5M
   400K .......... .......... .......... .......... ..........  689K
   450K .......... .......... .......... .......... .......... 9.04M
   500K .......... .

DR lc for object Mrk876 in band g saved


--2023-02-16 21:50:18--  https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE%20243.4882407%2065.719442%200.000277778&BANDNAME=r&FORMAT=csv
Resolving irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)... 134.4.54.110
Connecting to irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)|134.4.54.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./ZTF_special/Mrk876_DR_rband.csv’

     0K .......... .......... .......... .......... ..........  313K
    50K .......... .......... .......... .......... ..........  436K
   100K .......... .......... .......... .......... .......... 24.1M
   150K .......... .......... .......... .......... ..........  203K
   200K .......... .......... .......... .......... .......... 87.5M
   250K .......... .......... .......... .......... ..........  102M
   300K .......... .......... .......... .......... .......... 80.4M
   350K .......... .......... .......... .......... .......... 86.7M
 

...

HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./ZTF_special/H2106-099_DR_gband.csv’

     0K .......... .......... .......... .......... ..........  154K
    50K .......... ...                                          196K=0.4s

2023-02-16 21:51:46 (162 KB/s) - ‘./ZTF_special/H2106-099_DR_gband.csv’ saved [65471]

--2023-02-16 21:51:46--  https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE%20317.2915353%20-9.6707429%200.000277778&BANDNAME=r&FORMAT=csv
Resolving irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)... 134.4.54.110
Connecting to irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)|134.4.54.110|:443... 

DR lc for object H2106-099 in band g saved


connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./ZTF_special/H2106-099_DR_rband.csv’

     0K .......... .......... .......... ........

DR lc for object H2106-099 in band r saved


.. ..........  225K
    50K .......... .......... .......... ...                    227K=0.4s

2023-02-16 21:51:50 (226 KB/s) - ‘./ZTF_special/H2106-099_DR_rband.csv’ saved [85098]

